In [ ]:
import os
import sys
sys.path.append('/home/rstudio/codes')

In [ ]:
from pyBatman.nbrun import run_notebook
from pyBatman import load_config, get_db_path, sub_dir_path, mkdir_p
from pyBatman.constants import WD, SPECTRA_DIR, BACKGROUND_DIR, OUTPUT_DIR

In [ ]:
nb_name_input = '/home/rstudio/codes/notebooks/predict_spectra_concentrations.ipynb'

In [ ]:
if os.path.isdir(WD): # if wd exists (has been mapped)

    spectra_found = os.path.isdir(SPECTRA_DIR)
    background_found = os.path.isdir(BACKGROUND_DIR)
    
    if spectra_found and background_found:        
        input_spectra = sub_dir_path(SPECTRA_DIR)
        mkdir_p(OUTPUT_DIR)                
        for sd in sorted(input_spectra):

            print 'Now processing %s' % sd
            base_name = os.path.basename(os.path.normpath(sd))
            nb_name_output = os.path.join(OUTPUT_DIR, '%s.html' % base_name)
            
            timeout = -1 # never times out
            nb_kwargs = {'spectra_dir': sd}
            try:
                run_notebook(nb_name_input, nb_name_output, timeout=timeout, nb_kwargs=nb_kwargs)
            except Exception:
                continue

In [ ]:
import pandas as pd
import glob
import os
from IPython.display import display, HTML

In [ ]:
def get_file_id(csv_filename):
    basename = os.path.basename(os.path.normpath(csv_filename))
    filename_no_extension = os.path.splitext(basename)[0]
#     tokens = filename_no_extension.split('.')
#     file_id = tokens[1]
#     file_id = file_id.replace('ID_', '')
#     return file_id
    return filename_no_extension

In [ ]:
def get_df(csv_filename):
    df = pd.read_csv(csv_filename, header=0, index_col=0)
    file_id = get_file_id(csv_filename)
    df.rename(columns={'Concentration (mM)': file_id}, inplace=True)
    return df

In [ ]:
csv_dir = os.path.join(OUTPUT_DIR, 'csv')
csv_files = glob.glob(os.path.join(csv_dir, '*.csv'))
print '\n'.join(csv_files)

In [ ]:
dfs = []
for csv_filename in csv_files:
    df = get_df(csv_filename)
    display(df.transpose())
    dfs.append(df)

In [ ]:
combined_df = pd.concat(dfs, axis=1)
combined_df = combined_df.transpose()
display(combined_df)

In [ ]:
combined_df.to_csv(os.path.join(OUTPUT_DIR, 'results.csv'))